# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`


In [47]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [48]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [49]:
# confirm that our new database was created
mongo.list_database_names()

['admin',
 'autosaurus',
 'classDB',
 'config',
 'epa',
 'local',
 'met',
 'petsitly_marketing',
 'travel_db',
 'uk_food']

In [50]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [51]:
# review the collections in our new database
collections = db.list_collection_names()
collections

['establishments', 'Establishments']

In [52]:
# review a document in the establishments collection
db.establishments.find_one()

{'_id': ObjectId('64fa219e10ba4d63d162abd9'),
 'name': 'Penang Flavours',
 'type': 'Halal',
 'location': 'Greenwich',
 'rating': None,
 'geocode': {'longitude': None, 'latitude': None},
 'RatingValue': None}

In [53]:
# assign the collection to a variable
establishments = db['Establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [54]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "name": "Penang Flavours",
    "type": "Halal",
    "location": "Greenwich",
    "rating": None
}


In [55]:
# Insert the new restaurant into the collection
insert_result = establishments.insert_one(new_restaurant)

In [56]:
# Check that the new restaurant was inserted
if insert_result.acknowledged:
    print("Penang Flavours has been successfully added to the database.")
else:
    print("There was an issue adding Penang Flavours to the database.")

Penang Flavours has been successfully added to the database.


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [57]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}

# Projection to include only the desired fields
projection = {"BusinessTypeID": 1, "BusinessType": 1}

# Find the document that matches the query and project only the specified fields
result = db.establishments.find_one(query, projection)

if result:
    # Print the BusinessTypeID and BusinessType
    print("BusinessTypeID:", result["BusinessTypeID"])
    print("BusinessType:", result["BusinessType"])
else:
    print("No matching document found for 'Restaurant/Cafe/Canteen'.")

BusinessTypeID: 
BusinessType: Restaurant/Cafe/Canteen


3. Update the new restaurant with the `BusinessTypeID` you found.

In [58]:
# Update the new restaurant with the correct BusinessTypeID
establishments.insert_one({
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":""
})

In [59]:
# Confirm that the new restaurant was updated
# Retrieve the updated document
updated_document = establishments.find_one({"BusinessName": "Penang Flavours"})

# Check if the update was successful
if updated_document and updated_document.get("BusinessTypeID") == "1":
    print("Penang Flavours' BusinessTypeID has been successfully updated to 1.")
else:
    print("Penang Flavours' BusinessTypeID was not updated to 1, or the document was not found.")
print(updated_document)

Penang Flavours' BusinessTypeID was not updated to 1, or the document was not found.
{'_id': ObjectId('64fa22b510ba4d63d162abdd'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': ''}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [60]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_document_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of documents with LocalAuthorityName as 'Dover': {dover_document_count}")

Number of documents with LocalAuthorityName as 'Dover': 994


In [61]:
# Remove establishments within the Dover Local name
delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})

In [62]:
# Check if any remaining documents include Dover
remaining_documents_count = establishments.count_documents({})

print(f"Number of remaining documents after deletion: {remaining_documents_count}")

Number of remaining documents after deletion: 38787


In [65]:
# Check that other documents remain with 'find_one'
db.Establishments.find_one()

{'_id': ObjectId('64fa215389ecd2ff21ce6b7d'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [67]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [ {'$set':{ "geocode.longitude" : {'$toDecimal': "$geocode.longitude"},
                                           "geocode.latitude" : {'$toDecimal': "$geocode.latitude"}
                                         }} ])

Use `update_many` to convert `RatingValue` to integer numbers.

In [68]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [69]:
establishments.update_many({}, [ {'$set':{ "RatingValue" : {'$toInt': "$RatingValue"}}} ])

In [70]:
# Check that the coordinates and rating value are now numbers
print(establishments.find_one())

{'_id': ObjectId('64fa215389ecd2ff21ce6b7d'), 'FHRSID': 1043695, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000073616', 'BusinessName': 'The Pavilion', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'East Cliff Pavilion', 'AddressLine2': 'Wear Bay Road', 'AddressLine3': 'Folkestone', 'AddressLine4': 'Kent', 'PostCode': 'CT19 6BL', 'Phone': '', 'RatingValue': 5, 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2018-04-04T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': Decimal128('1.195625'), 'latitude': Decimal128('51.083812')}, 'RightToReply': '', 'Distance': 4591.765489457773, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00

In [71]:
pprint(establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64fa215389ecd2ff21ce6b7d'),
 'geocode': {'latitude': Decimal128('51.083812'),
             'longitude': Decimal128('1.195625')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel':